In [1]:
import requests as req
import psycopg2 as pg2
import datetime as dt

In [2]:
# Исходные данные

access_key = 'f6df1609cd905120e432af4619144713'
start_date = '2023-09-01'
end_date = '2023-09-30'
source = 'RUB'
currencies = 'BTC, USD, EUR'

currencies_list = currencies.replace(' ','').split(',')
start_date_dt = dt.date.fromisoformat(start_date)
end_date_dt = dt.date.fromisoformat(end_date)

In [3]:
# Запрос к Web API

url = 'http://api.exchangerate.host/timeframe?access_key={}&start_date={}&end_date={}&'\
    'source={}&currencies={}'.format(access_key, start_date, end_date, source, currencies)

response = req.get(url)
print('Статус-код: {}'.format(response.status_code))
data = response.json()

Статус-код: 200


In [4]:
# Создание таблиц

pg_hostname = 'db'
pg_port = '5432'
pg_username = 'postgres'
pg_pass = 'postgres'
pg_db = 'quotes'

conn = pg2.connect(host=pg_hostname, port=pg_port, user=pg_username, password=pg_pass, database=pg_db)
cursor = conn.cursor()

for cur in currencies_list:
    cursor.execute('CREATE TABLE IF NOT EXISTS {}('\
               'id SERIAL PRIMARY KEY,'\
               'quote_date date,'\
               'source VARCHAR(4),'\
               'currency VARCHAR(4),'\
               'rate float);'.format(source + cur)
              )

conn.commit()
cursor.close()
conn.close()

In [5]:
# Наполнение таблиц

conn = pg2.connect(host=pg_hostname, port=pg_port, user=pg_username, password=pg_pass, database=pg_db)
cursor = conn.cursor()

for cur in currencies_list:
    
    date_dt = start_date_dt
    cur_pair = source + cur
    insert_str = 'INSERT INTO {} (quote_date,source,currency,rate) VALUES'.format(cur_pair)

    while date_dt <= end_date_dt:
        date_str = str(date_dt)
        rate = data['quotes'][date_str][cur_pair]
        if rate > 0: rate = 1.0/rate
        else: rate = 'NULL'
        insert_str += "('{}','{}','{}',{}),".format(date_str, source, cur, str(rate))
        date_dt += dt.timedelta(days=1)
    
    insert_str = insert_str[:-1] + ';'
    cursor.execute(insert_str)

conn.commit()
cursor.close()
conn.close()

In [6]:
# Создание витрины

# LIMIT 1 в подзапросах нужен потому, что если они возвращают более одного значения, 
# возникает ошибка - при задвоении данных, или когда минимум/максимум приходится на несколько дат.

query = """(
WITH CTE AS(
SELECT
  MAX(rate) AS max_rate,
  MIN(rate) AS min_rate,
  AVG(rate) AS avg_rate,
  MAX(quote_date) AS last_day
FROM public.rubbtc)
SELECT
  'BTC' AS currency,
  (SELECT quote_date FROM public.rubbtc WHERE rate = (SELECT max_rate FROM CTE) LIMIT 1) AS max_rate_date,
  (SELECT quote_date FROM public.rubbtc WHERE rate = (SELECT min_rate FROM CTE) LIMIT 1) AS min_rate_date,
  (SELECT ROUND(max_rate::numeric,2) FROM CTE) AS max_rate,
  (SELECT ROUND(min_rate::numeric,2) FROM CTE) AS min_rate,
  (SELECT ROUND(avg_rate::numeric,2) FROM CTE) AS avg_rate,
  (SELECT ROUND(rate::numeric,2) FROM public.rubbtc WHERE quote_date = (SELECT last_day FROM CTE) LIMIT 1) AS last_day_rate
)
UNION ALL
(
WITH CTE AS(
SELECT
  MAX(rate) AS max_rate,
  MIN(rate) AS min_rate,
  AVG(rate) AS avg_rate,
  MAX(quote_date) AS last_day
FROM public.rubeur)
SELECT
  'EUR' AS currency,
  (SELECT quote_date FROM public.rubeur WHERE rate = (SELECT max_rate FROM CTE) LIMIT 1) AS max_rate_date,
  (SELECT quote_date FROM public.rubeur WHERE rate = (SELECT min_rate FROM CTE) LIMIT 1) AS min_rate_date,
  (SELECT ROUND(max_rate::numeric,2) FROM CTE) AS max_rate,
  (SELECT ROUND(min_rate::numeric,2) FROM CTE) AS min_rate,
  (SELECT ROUND(avg_rate::numeric,2) FROM CTE) AS avg_rate,
  (SELECT ROUND(rate::numeric,2) FROM public.rubeur WHERE quote_date = (SELECT last_day FROM CTE) LIMIT 1) AS last_day_rate
)
UNION ALL
(
WITH CTE AS(
SELECT
  MAX(rate) AS max_rate,
  MIN(rate) AS min_rate,
  AVG(rate) AS avg_rate,
  MAX(quote_date) AS last_day
FROM public.rubusd)
SELECT
  'USD' AS currency,
  (SELECT quote_date FROM public.rubusd WHERE rate = (SELECT max_rate FROM CTE) LIMIT 1) AS max_rate_date,
  (SELECT quote_date FROM public.rubusd WHERE rate = (SELECT min_rate FROM CTE) LIMIT 1) AS min_rate_date,
  (SELECT ROUND(max_rate::numeric,2) FROM CTE) AS max_rate,
  (SELECT ROUND(min_rate::numeric,2) FROM CTE) AS min_rate,
  (SELECT ROUND(avg_rate::numeric,2) FROM CTE) AS avg_rate,
  (SELECT ROUND(rate::numeric,2) FROM public.rubusd WHERE quote_date = (SELECT last_day FROM CTE) LIMIT 1) AS last_day_rate
)
"""

conn = pg2.connect(host=pg_hostname, port=pg_port, user=pg_username, password=pg_pass, database=pg_db)
cursor = conn.cursor()
cursor.execute(query)
conn.commit()
res = cursor.fetchall()
cursor.close()
conn.close()

print()
print('cur   max_rate_dt   min_rate_dt   max_rate     min_rate     avg_rate     last_d_rate')
print('------------------------------------------------------------------------------------')
for i in range(len(currencies_list)):
    print('{}   {}    {}    {:10.2f}   {:10.2f}   {:10.2f}   {:10.2f}'.format(*res[i]))


cur   max_rate_dt   min_rate_dt   max_rate     min_rate     avg_rate     last_d_rate
------------------------------------------------------------------------------------
BTC   2023-09-19    2023-09-11    2654195.49   2392207.15   2542285.00   2643509.52
EUR   2023-09-07    2023-09-25        105.37       100.45       103.22       103.75
USD   2023-09-07    2023-09-25         98.50        94.84        96.64        98.00
